In [1]:
import pickle

from functions.cleaning import cleaning_rf
from functions.preprocessor import preprocessor

from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV
from sklearn.ensemble import RandomForestClassifier

from imblearn.pipeline import Pipeline
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

# Import of clean data

In [2]:
df = cleaning_rf()

In [3]:
X = df.drop(columns=['EVENT_LABEL'])
y = df['EVENT_LABEL']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=0, stratify=y)

In [4]:
df.head()

,transaction_amt,transaction_adj_amt,historic_velocity,currency,cvv,signature_image,transaction_type,transaction_env,tranaction_initiate,inital_amount,EVENT_LABEL,day,month,browser,os,acc_age,d_last_logon
0,2167.0,56.0,2572.0,cad,D,F,U,X,O,13646.0,0,Tue,Dec,Opera,Windows,4,3
1,2045.0,48.0,4517.0,cad,X,X,H,W,J,11930.0,0,Tue,Jun,Opera,Linux,3,1
2,2892.0,61.0,5007.0,cad,X,Q,X,X,T,7412.0,0,Mon,May,Mozilla,Linux,7,5
3,3040.0,28.0,6022.0,usd,G,G,C,N,M,4623.0,0,Thu,Mar,Mozilla,Macintosh,3,2
4,2976.0,66.0,2600.0,cad,X,F,F,G,K,1905.0,0,Sat,Mar,Mozilla,Linux,4,2


## Preprocessor

In [ ]:
preprocessor = preprocessor(X_train)

## Model

In [ ]:
RandomForest = RandomForestClassifier(n_jobs=-1, random_state=42)

# Random forest classifier

## Undersampling

### Under sampler

In [5]:
under_sampler = RandomUnderSampler()

### Pipeline

In [7]:
u_rf = Pipeline([('under_sampler', under_sampler),
                 ('preprocesor', preprocessor),
                 ('RandomForest', RandomForest)])

In [8]:
results = cross_validate(u_rf, X_train, y_train,cv=5, return_train_score = True, scoring='recall_macro')
print(f'Average Train Score: {results['train_score'].mean()}')
print(f'Average Test Score: {results['test_score'].mean()}')

/Users/jakubkozlowski/anaconda3/envs/data_mining/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:227: UserWarning: Found unknown categories in columns [1, 3, 4] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/jakubkozlowski/anaconda3/envs/data_mining/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:227: UserWarning: Found unknown categories in columns [1, 2, 3, 4] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/jakubkozlowski/anaconda3/envs/data_mining/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:227: UserWarning: Found unknown categories in columns [2, 3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/jakubkozlowski/anaconda3/envs/data_mining/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:227: UserWarning: Found unknown categories in columns [1, 2, 3, 4] during transform. Thes

Average Train Score: 0.9635410400705183
Average Test Score: 0.8940145263696397


In [9]:
u_grid = {'RandomForest__max_depth': [5, 8, 10, 15],
          'RandomForest__max_features': [3, 6, 9, 13],
          'RandomForest__min_samples_split': [4, 8, 10],
          'RandomForest__min_samples_leaf': [2, 4, 8, 10]}

In [10]:
u_grid_search = GridSearchCV(estimator=u_rf, param_grid=u_grid, cv=5, scoring='recall_macro', return_train_score=True)
u_grid_search.fit(X_train, y_train)
print(f'Average Train Score: {u_grid_search.cv_results_['mean_train_score'].mean()}')
print(f'Average Test Score: {u_grid_search.cv_results_['mean_test_score'].mean()}')

/Users/jakubkozlowski/anaconda3/envs/data_mining/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:227: UserWarning: Found unknown categories in columns [1, 2, 3, 4] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/jakubkozlowski/anaconda3/envs/data_mining/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:227: UserWarning: Found unknown categories in columns [1, 2, 3, 4] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/jakubkozlowski/anaconda3/envs/data_mining/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:227: UserWarning: Found unknown categories in columns [2, 3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/jakubkozlowski/anaconda3/envs/data_mining/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:227: UserWarning: Found unknown categories in columns [1, 2, 3, 4] during transform. T

KeyboardInterrupt: 

In [ ]:
u_grid_search.best_estimator_

Pipeline(steps=[('under_sampler', RandomUnderSampler()),
                ('preprocesor',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore'),
                                                  Index(['currency', 'cvv', 'signature_image', 'transaction_type',
       'transaction_env', 'tranaction_initiate', 'day', 'month', 'browser',
       'os'],
      dtype='object')),
                                                 ('num', StandardScaler(),
                                                  Index(['transaction_amt', 'transaction_adj_amt', 'historic_velocity',
       'inital_amount', 'acc_age', 'd_last_logon'],
      dtype='object'))])),
                ('RandomForest',
                 RandomForestClassifier(max_depth=9, max_features=13,
                                        min_samples_split=6, n_jobs=-1,
                                        random_state=42))])

## Oversampling

### Over sampler

In [ ]:
over_sampler = RandomOverSampler()

### Pipeline

In [ ]:
o_rf = Pipeline([('over_sampler', over_sampler),
                 ('preprocessor', preprocessor),
                 ('RandomForest', RandomForest)])

In [ ]:
results = cross_validate(o_rf, X_train, y_train, return_train_score = True, scoring='recall_macro')
print(f'Average Train Score: {results['train_score'].mean()}')
print(f'Average Test Score: {results['test_score'].mean()}')

/Users/jakubkozlowski/anaconda3/envs/data_mining/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:227: UserWarning: Found unknown categories in columns [1, 4] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Average Train Score: 1.0
Average Test Score: 0.8140728458596576


In [ ]:
o_grid = {'RandomForest__max_depth': [5, 8, 10, 15],
          'RandomForest__max_features': [3, 6, 9, 13],
          'RandomForest__min_samples_split': [4, 8, 10],
          'RandomForest__min_samples_leaf': [2, 4, 8, 10]}

In [ ]:
o_grid_search = GridSearchCV(estimator=o_rf, param_grid=o_grid, cv=3, scoring='recall_macro', return_train_score=True)
o_grid_search.fit(X_train, y_train)
print(f'Average Train Score: {o_grid_search.cv_results_['mean_train_score'].mean()}')
print(f'Average Test Score: {o_grid_search.cv_results_['mean_test_score'].mean()}')

/Users/jakubkozlowski/anaconda3/envs/data_mining/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:227: UserWarning: Found unknown categories in columns [1, 4] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/jakubkozlowski/anaconda3/envs/data_mining/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:227: UserWarning: Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/jakubkozlowski/anaconda3/envs/data_mining/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:227: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/jakubkozlowski/anaconda3/envs/data_mining/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:227: UserWarning: Found unknown categories in columns [1, 4] during transform. These unknown categories 

Average Train Score: 0.8836532882855567
Average Test Score: 0.8710384115609431


In [ ]:
o_grid_search.best_estimator_

Pipeline(steps=[('over_sampler', RandomOverSampler()),
                ('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore'),
                                                  Index(['currency', 'cvv', 'signature_image', 'transaction_type',
       'transaction_env', 'tranaction_initiate', 'day', 'month', 'browser',
       'os'],
      dtype='object')),
                                                 ('num', StandardScaler(),
                                                  Index(['transaction_amt', 'transaction_adj_amt', 'historic_velocity',
       'inital_amount', 'acc_age', 'd_last_logon'],
      dtype='object'))])),
                ('RandomForest',
                 RandomForestClassifier(max_depth=15, max_features=13,
                                        min_samples_leaf=8, min_samples_split=8,
                                        n_jobs=-1, random_state=42))])

# Export

In [ ]:
with open('models/under_rf.pkl', 'wb') as f:
    pickle.dump(u_grid_search, f)
f.close()

In [ ]:
with open('models/over_rf.pkl', 'wb') as f:
    pickle.dump(o_grid_search, f)
f.close()